In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# # The GPU id to use, usually either "0" or "1";
# os.environ["CUDA_VISIBLE_DEVICES"]="0";

# TensorFlow and tf.keras
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.15.0


Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:3 -> device: 3, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1



In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# the data, split between train and test sets
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(y_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(4, 4),
                 strides=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (4, 4), strides=1, activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

W0121 15:26:43.146782 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0121 15:26:43.149754 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0121 15:26:43.153221 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [7]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

W0121 15:26:44.218905 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0121 15:26:44.226702 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

W0121 15:26:45.661915 140435593516864 deprecation.py:323] From /home/jiameng/packages/Bernsp/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0121 15:26:45.769882 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0121 15:26:45.774112 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0121 15:26:45.905589 140435593516864 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 82us/step - loss: 0.5754 - acc: 0.8587 - val_loss: 0.1242 - val_acc: 0.9692
Epoch 2/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.0973 - acc: 0.9730 - val_loss: 0.0767 - val_acc: 0.9774
Epoch 3/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.0602 - acc: 0.9825 - val_loss: 0.0514 - val_acc: 0.9849
Epoch 4/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.0434 - acc: 0.9873 - val_loss: 0.0442 - val_acc: 0.9865
Epoch 5/12
60000/60000 [==============================] - 3s 52us/step - loss: 0.0329 - acc: 0.9906 - val_loss: 0.0388 - val_acc: 0.9883
Epoch 6/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.0254 - acc: 0.9927 - val_loss: 0.0380 - val_acc: 0.9879
Epoch 7/12
60000/60000 [==============================] - 3s 50us/step - loss: 0.0194 - acc: 0.9949 - val_loss: 0.0386 - val_acc

In [9]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 1s 51us/step
Test accuracy: 0.9889


## Make predictions

In [10]:
predictions = model.predict(x_test)

In [11]:
predictions[0]

array([2.5129609e-07, 6.1050973e-06, 3.1777238e-06, 4.3679997e-06,
       1.6736908e-07, 8.2095687e-07, 2.1970196e-08, 9.9997735e-01,
       3.8860924e-07, 7.3203405e-06], dtype=float32)

In [12]:
np.argmax(predictions[0])

7

In [13]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [14]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_MNIST_CNN_Small.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_MNIST_CNN_Small.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
weights = []
for layer in model.layers:
    weights.append(layer.get_weights()) # list of numpy arrays

In [16]:
model.layers[0].get_weights()[0].shape

(4, 4, 1, 16)